In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import csv
import collections
import dateutil.parser as date
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import sys
import sqlite3 as lite
from matplotlib import cm
from pyne import nucname as nn
from IPython.display import Image, HTML, display
import seaborn as sns 

sys.path.append('../../../scripts/')
import analysis as an

In [18]:
# Before this step, must download the output sqlite files from the fuel-cycles box. 
# fuel-cycle/cyclus_output/predicting_the_past_validation 
# and put into a directory named cyclus 

def get_cursor(file_name):
    """ Connects and returns a cursor to an sqlite output file

    Parameters
    ----------
    file_name: str
        name of the sqlite file

    Returns
    -------
    sqlite cursor
    """
    con = lite.connect(file_name)
    con.row_factory = lite.Row
    return con.cursor()


cursor = get_cursor('cyclus/united_states_decayon.sqlite')

In [19]:
def get_timesteps(cur):
    """ Returns simulation start year, month, duration and
    timesteps (in numpy linspace).

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    init_year: int
        start year of simulation
    init_month: int
        start month of simulation
    duration: int
        duration of simulation
    timestep: list
        linspace up to duration
    """
    info = cur.execute('SELECT initialyear, initialmonth, '
                       'duration FROM info').fetchone()
    init_year = info['initialyear']
    init_month = info['initialmonth']
    duration = info['duration']
    timestep = np.linspace(0, duration - 1, num=duration)

    return init_year, init_month, duration, timestep

In [24]:
def spent_fuel_produced(cur): 
    """ Find timeseries of mass of spent fuel output by reactors 
    
    Parameters 
    ----------
    cur: sqlite cursor 
    
    Returns 
    -------
    timeseries list of spent fuel output by reactors
    """

    init_yr, init_month, duration, timestep = get_timesteps(cursor)
    spent_fuel = cur.execute('SELECT time, sum(quantity) FROM transactions '
                             'INNER JOIN resources '
                             'ON transactions.resourceid = resources.resourceid '
                             'WHERE Commodity = "cool_spent_uox3" '
                             'GROUP BY time ').fetchall()
    spent_fuel_list = an.get_timeseries(spent_fuel,duration,True)
    spent_fuel_total = np.sum(spent_fuel_list)
    return spent_fuel_total 

total_spentfuel = spent_fuel_produced(cursor)

In [25]:
total_spentfuel

7679.995986667509

In [43]:
def isotope_total(cur):
    """ Find list of isotopes present in spent fuel output by reactor 
    
    Parameters 
    ----------
    cur: sqlite cursor 
    
    Returns 
    -------
    list of isotopes in spent fuel output by reactors
    """    
    
    init_yr, init_month, duration, timestep = get_timesteps(cursor)
    isotopes = cur.execute('SELECT time, sum(quantity)*massfrac FROM transactions '
                           'INNER JOIN resources '
                           'ON transactions.resourceid = resources.resourceid '
                           'LEFT OUTER JOIN compositions '
                           'ON resources.qualid = compositions.qualid '
                           'WHERE Commodity = "cool_spent_uox9" AND nucid = 942390000 '
                           ' GROUP BY time ').fetchall()
    isotope_list = an.get_timeseries(isotopes,duration,True)
    isotope_total = np.sum(isotope_list)
    return isotope_total

#total_U238 = isotope_total(cursor) # in MTHM 
total_Pu239_9 = isotope_total(cursor)

In [50]:
Pu239_array = [total_Pu239, total_Pu239_2, total_Pu239_3, total_Pu239_4, total_Pu239_5, total_Pu239_6, total_Pu239_7, total_Pu239_8, total_Pu239_9]
Pu239_array_cum = np.cumsum(Pu239_array)

In [51]:
Pu239_array_cum

array([  9.35330724,  47.85605458,  97.07469715, 162.16246537,
       207.01699025, 253.3411486 , 299.50518161, 343.75024945,
       381.35100507])